In [1]:
import sys
sys.path.append("..")
import tensorflow as tf
import input_data
import numpy as np


print(tf.__version__)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

1.12.0


In [2]:
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(layer, W):
    return tf.nn.conv2d(layer, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pooling_2x2(layer):
    return tf.nn.max_pool(layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
W_conv1 = weight_variable([5, 5, 1, 16]) # RGB: Channel = 3
b_conv1 = bias_variable([16])

W_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pooling_2x2(h_conv1)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pooling_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 32])

W_fc1 = weight_variable([7 * 7 * 32, 256])
b_fc1 = bias_variable([256])

W_fc2 = weight_variable([256, 10])
b_fc2 = bias_variable([10])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
y = tf.nn.softmax(h_fc2)

In [5]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [6]:
# Train
training_iteration = 10000
batch_size = 50
display_step = 100
tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./cnn/', sess.graph)
    sess.run(tf.initialize_all_variables())
    for iteration in range(training_iteration):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        summary, _, current_accuracy = sess.run([merged, train_step, accuracy], feed_dict={x: batch_xs, y_: batch_ys})
        writer.add_summary(summary, iteration)
        if iteration % display_step == 0:
            print('Iteration: %5d | Accuracy = %.6f' % (iteration + 1, current_accuracy))
    print('Test accuracy: %.6f' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    writer.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iteration:     1 | Accuracy = 0.160000
Iteration:   101 | Accuracy = 0.720000
Iteration:   201 | Accuracy = 0.860000
Iteration:   301 | Accuracy = 0.820000
Iteration:   401 | Accuracy = 0.840000
Iteration:   501 | Accuracy = 0.800000
Iteration:   601 | Accuracy = 0.820000
Iteration:   701 | Accuracy = 0.860000
Iteration:   801 | Accuracy = 0.840000
Iteration:   901 | Accuracy = 0.860000
Iteration:  1001 | Accuracy = 0.900000
Iteration:  1101 | Accuracy = 0.840000
Iteration:  1201 | Accuracy = 0.840000
Iteration:  1301 | Accuracy = 0.760000
Iteration:  1401 | Accuracy = 0.880000
Iteration:  1501 | Accuracy = 0.860000
Iteration:  1601 | Accuracy = 0.860000
Iteration:  1701 | Accuracy = 0.900000
Iteration:  1801 | Accuracy = 0.840000
Iteration:  1901 | Accuracy = 0.820000
Iteration:  2001 | Accuracy = 0.920000
Iteration:  2101 | Accuracy = 0.820000
Iteration:  2201 | Accuracy = 0.880000
Iteration:  2301 | Accuracy =